### 1.Импорты

In [10]:
import csv
import os 
import pandas as pd

In [ ]:
PATH = "data"

pd.set_option('display.width', 1000) # Для того чтобы выводилась таблица без переносов

# Используем os.path.join для кроссплатформенности, так как в linux и windows по разному строятся пути к файлам
FILETEST = os.path.join(PATH, "test.csv")    
FILETRAIN = os.path.join(PATH, "train.csv") 

try:
    df_train = pd.read_csv(FILETRAIN)
    df_test = pd.read_csv(FILETEST)
    print("Данные успешно загружены!")
    print("\nПервые 5 строк тренировочного набора:")
    print(df_train.head())
    print(f"\nРазмер тренировочного набора: {df_train.shape}")
    print(f"Размер тестового набора: {df_test.shape}")
except FileNotFoundError:
    print(f"Ошибка: Один из файлов не найден. Проверьте пути:\n- {FILETRAIN}\n- {FILETEST}")
except Exception as e:
    print(f"Произошла ошибка при загрузке данных: {e}")


Данные успешно загружены!

Первые 5 строк тренировочного набора:
   PassengerId  Survived  Pclass                                               Name     Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked
0            1         0       3                            Braund, Mr. Owen Harris    male  22.0      1      0         A/5 21171   7.2500   NaN        S
1            2         1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1      0          PC 17599  71.2833   C85        C
2            3         1       3                             Heikkinen, Miss. Laina  female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S
3            4         1       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1      0            113803  53.1000  C123        S
4            5         0       3                           Allen, Mr. William Henry    male  35.0      0      0            373450   8.0500   NaN        S

Размер тре